In [ ]:
## import the postgres connector
import psycopg2

## INSERT THE PASSWORD

## establishing the connection
conn = psycopg2.connect(
   database="test_db", user='shoc', password='JustKeepSwimming', host='pg_container', port= '5432'
)

## Creating a cursor object using the cursor() method
cursor = conn.cursor()

## Executing an SQL function using the execute() method
cursor.execute("select version()")

## Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)

In [ ]:
cursor = conn.cursor()

## Executing an SQL function using the execute() method
cursor.execute("select * from loki_aggregated")

## Fetch a single row using fetchone() method.
data = cursor.fetchall()
print("Pulled all of the data in loki_aggregated: ", type(data), " of length ", len(data))

## Closing the connection
conn.close()

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from re import search

## Make a dataframe from data
df = pd.DataFrame(data)

## For some reason when we pull from postgres we don't get column
##     names, so go ahead and add them back
df.columns =['container id', 'role', 'user name', 'process', 'message start', 'message stop', 'start time', 'stop time']

## I wrote a custom
def convertStringToDateTime(cell):
    tmp = cell.to_string(header=False, index=False)
    if tmp == 'NaT':
        return None
    
    try:
        return datetime.strptime(tmp, '%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        return datetime.strptime(tmp, '%Y-%m-%d %H:%M:%S')


    
## Add another column for our df that calculates the diffence 
##     between the start and stop time
timeDelta = []
for i in range(0, len(df.index)):
    start = convertStringToDateTime(df.loc[i, ['start time']])
    stop = convertStringToDateTime(df.loc[i, ['stop time']])
    
    if start == None or stop == None:
        timeDelta.append(None)
    else:
        timeDelta.append((stop - start).total_seconds() / 60)
    
## Now we are done add the time take column
df['delta'] = timeDelta
df

In [ ]:
## Make a bar graph of the time delta vs process names
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 6))
ax.bar(df['process'], df['delta'],
       label='Process time')
ax.legend()

plt.xticks(rotation=10, wrap=True)
plt.show()

In [ ]:
## Make a simple bar graph of the avg time delta
df_means = df.groupby('process').mean()
mean = df['delta'].mean()
median = df['delta'].median()

fig, ax = plt.subplots(figsize=(20, 6))
ax.bar(df_means.index, df_means['delta'],
       label='Process Time')

ax.axhline(mean, 0, 1, color='red', label='Mean Value for Process Time')
ax.axhline(median, 0, 1, color='green', label='Median Value for Process Time')
ax.legend()

plt.xticks(rotation=10, wrap=True)
plt.show()